In [4]:
# DWave Credentials
dwave_solver = "DW_2000Q_2_1"
api_key = 'DEV-3142f58be13a0aacd44657b15b8fb9832ba3bdac'

In [9]:
# Import correlation data
import scipy.io as sio

P_array = sio.loadmat('P_mat.mat').get('P_mat')
A_array = sio.loadmat('A_mat.mat').get('A_mat')
S_array = sio.loadmat('S_mat.mat').get('S_mat')

In [33]:
# Import graph node labels
node_labels = []
with open('CryptoSymbols.csv') as f:
    for line in f.readlines():
        line = line.rstrip()
        node_labels.append(line)

In [26]:
graph_250 = S_array[250,:,:]
graph_250.shape

(27, 27)

In [29]:
# Change correlation values to 0,+1,-1
for i in range(27):
    for j in range(27):
        if graph_250[i,j] < 0:
            graph_250[i,j] = int(1)
        elif graph_250[i,j] > 0:
            graph_250[i,j] = int(-1)

In [38]:
### Use NetworkX graphic package to create a signed network
import networkx as nx

G = nx.from_numpy_matrix(graph_250)

In [39]:
G.number_of_nodes()

27

In [5]:
# Select a D-Wave system and handle mapping from problem graph to sampler graph
from dwave.system.samplers import DWaveSampler
from dwave.system.composites import EmbeddingComposite

sampler = EmbeddingComposite(DWaveSampler(solver=dwave_solver, token=api_key))

In [6]:
# Return a good partition (minimal structural imbalance) and its frustrated edges  
import dwave_networkx as dnx
imbalance, bicoloring = dnx.structural_imbalance(G, sampler)

# Mark the returned frustrated edges and node set (color) on the graph 
for edge in G.edges:
    G.edges[edge]['frustrated'] = edge in imbalance
for node in G.nodes:
    G.nodes[node]['color'] = bicoloring[node]

# Print the relationships for the returned partition
print('Yellow set: ', [person for (person, color) in bicoloring.items() if (color == 0)])
print('Blue set: ', [person for (person, color) in bicoloring.items() if (color == 1)])
print('\nFrustrated relationships: ', list(imbalance.keys()))

Yellow set:  ['Alice', 'Bob', 'Wally']
Blue set:  ['Eve']

Frustrated relationships:  [('Alice', 'Wally')]
